## ASG5
          

Cluster the business sections (/blue/acg7849/share/BS) using Doc2Vec (50 clusters) in two ways:

Using a counter as the ‘tag’ (as in 5.1.5)

Using a counter as the ‘tag’, and the industry code as an additional tag (yield TaggedDocument(words=file_tokens, tags=[i, SIC]) where SIC is a string holding the tag (for example ‘1740’)

Extract the 4-digit SIC industry code from the annual report header (STANDARD INDUSTRIAL CLASSIFICATION).

Required: Evaluate whether adding the industry code as an additional tag improves the clustering. Use the standard deviation of profitability as a way to evaluate this. (Firms that are more similar, should have similar performance. Therefore, a better clustering would result in lower standard deviations for each cluster, relative to a worse clustering).

Do this for the filings for the year 2019 only. Calculate the standard deviation of performance for each cluster (use the year of CONFORMED END OF PERIOD, which are the first 4 digits of ‘date’ in summary.text).

For 50 clusters that means you will have 2 standard deviations for each cluster (one for each approach, with the extra SIC tag vs not adding the extra SIC tag). Use a t-test to test for a difference between the two sets of 50 standard deviations.

In [1]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import gensim
stopWords = set(stopwords.words('english') )
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk import FreqDist
from nltk.collocations import *

/apps/jupyterhub/1.1.0/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# already made this - copy + paste to personal directory

#try copy and past targets
#think this is a pretty bold way avoiding messing up with the class

import shutil
for i in range(len(listoflink)):
    shutil.copy(listoflink[i], r'/blue/acg7849/hli1/item1')
    #shutil.copy(listoflink[i], r'/blue/acg7849/tbode/whateverfile')

# codes from 515 - start with new directory

In [2]:
# add some punctuation to string.punctuation
punc = string.punctuation + '“”'

# documents get tagged by an index (number), while filenames have different numbers
# keep track of this
fileIdToIndex = {} # given a fileId -> tag
indexToFileId=[] # given a tag -> fileId

class MyFiles(object):
    def __init__(self, dirname, tokens_only = False):
        self.dirname = dirname
        self.tokens_only = tokens_only
 
    def __iter__(self):
        #for i, fname in enumerate(files[0:200]):
         
        for i, fname in enumerate(os.listdir(self.dirname)[0:200]):
        # enumerate = return a list of tuples, iterate from start to end, [0:200] must be there
        # os.listdir = return index of a directory, input = directory address
        # this part enumerates the first 200 units in the index under dirname
        
            with open( os.path.join(self.dirname, fname), encoding='utf-8') as f:
                content = f.read()
            # filter
                #content = [f for f in content if int(f["length"])> 1000 and (f["date"][0:4]) == '2019']
                #content = [f for f in content if len(f)> 1000]
            # grab id from filename
            myCounter = int ( re.findall(r'(\d*)\.txt', fname)[0] )
            # update 
            fileIdToIndex [ myCounter] = i
            indexToFileId.append( myCounter)
            #print('fname', fname, 'tag', myCounter)
            file_tokens = [x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation]
            
            if self.tokens_only == True:
                yield file_tokens
            else:
                yield TaggedDocument(words=file_tokens, tags=[i] )                    

In [3]:
# Hipergator
ffiles = MyFiles(r'/blue/acg7849/hli1/item1') # this one expected str, bytes or os.PathLike object
#ffiles = MyFiles(r'/blue/acg7849/share/BS/item1/') # a memory-friendly iterator
# dirname = '/blue/acg7849/share/BS/item1/'?

# create a model, build vocabulary
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=10)
model.build_vocab(ffiles) 
# train it
model.train(ffiles, total_examples=model.corpus_count, epochs=model.epochs)

In [4]:
# Hipergator
def tokenizeFile(file_id):
    with open( r'/blue/acg7849/hli1/item1/'+str(file_id)+'.txt', encoding='utf-8') as f:
            content = f.read()
    return ([x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation] )

In [5]:
t = tokenizeFile(277350) # test file = 277350
model.infer_vector( t )

array([-1.811593  ,  0.2716504 , -1.6636219 , -2.0849779 ,  3.5197663 ,
       -2.5536299 , -2.6030385 , -1.3686801 , -3.014339  ,  0.21578524,
       -0.23528711,  2.5992036 , -1.4219581 ,  0.98291737,  0.32460353,
        0.9410857 ,  0.92102945,  2.349767  , -1.6231432 , -5.011881  ,
       -0.65269136,  0.48169163, -0.05643673,  0.3300975 , -0.41524637,
       -0.29054004, -2.1311615 ,  0.36154285, -1.6587504 ,  0.9253874 ,
        0.2919745 ,  1.3644869 , -1.2212223 , -1.4661137 ,  0.7169397 ,
       -1.2937855 , -0.31634545, -3.0772135 ,  1.0376463 ,  0.16531998,
       -0.9418244 , -0.5249591 ,  1.666985  ,  2.3974621 , -0.34267205,
       -2.543288  ,  0.40126285,  2.3897433 ,  0.38184747,  0.13161495,
       -1.2011691 ,  1.4179391 ,  1.0272518 ,  0.9971536 , -2.0866113 ,
       -0.5776119 , -1.9123391 ,  1.6437991 , -0.6308292 ,  1.6901064 ,
        3.4923859 ,  0.1999571 ,  2.418181  , -1.1094514 , -1.3805083 ,
       -0.8175258 ,  0.98982745,  0.3472884 ,  0.5286452 ,  1.23

In [6]:
t = tokenizeFile(277350)
inferred_vector = model.infer_vector( t )
# dv is short for docvecs
# most similar files to 277350 is the 4th one 265065?
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
#sims = model.dv.most_similar([inferred_vector], topn=4)
sims[0:5]

[(3, 0.6917709708213806),
 (180, 0.689142107963562),
 (107, 0.6726787686347961),
 (26, 0.6559200882911682),
 (37, 0.611767590045929)]

In [7]:
# letter with filename 1.txt is the first letter, so tag is 0
similar_doc = model.docvecs.most_similar(0)
similar_doc

<ipython-input-7-0bceb90a8426>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(0)


[(103, 0.8206000328063965),
 (93, 0.7932485342025757),
 (149, 0.7799310088157654),
 (55, 0.7510608434677124),
 (198, 0.7487143278121948),
 (107, 0.7478072047233582),
 (115, 0.7472643852233887),
 (90, 0.7085216641426086),
 (37, 0.7024698853492737),
 (38, 0.6997923254966736)]

In [8]:
print('number of documents', model.corpus_count)
print('model.docvecs', len(model.dv))

number of documents 200
model.docvecs 200


In [9]:
# Hipergator
# reread the files, and get the vector for each file
# feed vector into k-means algorithm to make clusters
wordLists = MyFiles(r'/blue/acg7849/hli1/item1', tokens_only = True) # a memory-friendly iterator
vectors = [ model.infer_vector( w ) for w in wordLists]
len(vectors)
vectors[0]

array([ 0.8914902 ,  1.0365181 , -0.6279697 , -0.65517074,  1.962731  ,
       -0.57669824, -5.149925  , -0.21185449, -3.921839  , -3.2669826 ,
        1.4117745 ,  2.70079   , -0.72105616,  0.39709362, -2.2376409 ,
        0.27832437,  3.3653367 ,  3.3251123 , -2.647636  , -2.531493  ,
       -4.2508774 , -0.09885166, -0.10626055, -1.069596  ,  1.5419154 ,
       -3.2789903 , -1.6415857 ,  2.1012747 ,  0.32585385, -1.5470833 ,
        3.5130355 ,  1.2856466 , -1.6063392 , -0.99121356, -1.2053097 ,
       -1.7892944 ,  0.6532468 , -5.162125  ,  0.45646808, -0.74689907,
       -0.08406452, -1.5623255 ,  3.4916182 ,  0.447897  , -1.361903  ,
       -4.367096  , -0.6598568 ,  0.4116321 ,  1.7339507 ,  1.7754472 ,
        3.207348  ,  1.7356949 ,  0.9911989 , -0.49050486, -2.640037  ,
        2.4847753 , -1.0687668 ,  1.1662743 ,  1.2444549 ,  5.057472  ,
        4.4588814 , -0.09393429,  5.1969934 ,  0.481649  , -0.6853551 ,
       -1.8203465 , -2.1596158 ,  0.6645025 ,  0.46371752, -0.57

In [10]:
from nltk.cluster import KMeansClusterer
num_clusters = 10
kclusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)

In [11]:
import collections
print(collections.Counter(assigned_clusters))

Counter({6: 31, 4: 28, 1: 24, 9: 21, 0: 21, 8: 21, 2: 19, 7: 13, 3: 13, 5: 9})


# also needs: get industrial codes